In [3]:
import os
import sys

# 1. FIXER LES VARIABLES AVANT TOUT (Évite les conflits Protobuf/TensorFlow)
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["USE_TF"] = "0"
os.environ["USE_TORCH"] = "1"

# 2. VÉRIFICATION ET IMPORTS
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    from transformers import pipeline
    import torch
    print("✅ Environnement chargé avec succès !")
except Exception as e:
    print(f"❌ Erreur d'import : {e}")
    print("💡 Essayez : !pip install protobuf==3.20.3 transformers torch ipywidgets")

# 3. CONFIGURATION DES MODÈLES ET DU MAPPING DES LABELS
MODELS_CONFIG = {
    "🇫🇷 CamemBERT": "LamT45/camenbert_fakenews_model",
    "🇬🇧 BERT": "LamT45/ENG_Bert_fake_news_model_0301",
    "🇺🇸 RoBERTa": "LamT45/roberta-fake-news-ENG"
}

# Mapping inversé selon tes observations :
# CamemBERT : 1 = True, 0 = Fake
# BERT/RoBERTa : 1 = Fake, 0 = True
LABEL_MAPPINGS = {
    "🇫🇷 CamemBERT": {"LABEL_0": "🚨 Fake (Faux)", "LABEL_1": "✅ Reliable (Vrai)"},
    "🇬🇧 BERT":      {"LABEL_0": "✅ Reliable (Vrai)", "LABEL_1": "🚨 Fake (Faux)"},
    "🇺🇸 RoBERTa":   {"LABEL_0": "✅ Reliable (Vrai)", "LABEL_1": "🚨 Fake (Faux)"}
}

loaded_pipelines = {}

# --- INTERFACE GRAPHIQUE ---
model_dropdown = widgets.Dropdown(
    options=MODELS_CONFIG.keys(), 
    description='Modèle:', 
    style={'description_width': 'initial'}
)
text_input = widgets.Textarea(
    placeholder='Collez l\'article à analyser ici...', 
    layout={'width': '100%', 'height': '150px'}
)
analyze_button = widgets.Button(
    description='🚀 Analyser Localement', 
    button_style='success',
    layout={'width': '250px'}
)
output_area = widgets.Output()

# --- LOGIQUE DE PRÉDICTION ---
def get_local_prediction(model_key, text):
    model_id = MODELS_CONFIG[model_key]
    if model_id not in loaded_pipelines:
        with output_area:
            print(f"📥 Premier chargement de {model_id} (veuillez patienter)...")
        loaded_pipelines[model_id] = pipeline("text-classification", model=model_id)
    return loaded_pipelines[model_id](text)

def on_button_clicked(b):
    with output_area:
        clear_output()
        if not text_input.value.strip():
            print("⚠️ Veuillez saisir un texte avant de lancer l'analyse.")
            return
        
        try:
            nom_modele = model_dropdown.value
            res = get_local_prediction(nom_modele, text_input.value)
            
            # Extraction des données
            raw_label = res[0]['label']
            score = res[0]['score'] * 100
            
            # Application du mapping spécifique au modèle
            mapping = LABEL_MAPPINGS.get(nom_modele, {})
            friendly_label = mapping.get(raw_label, raw_label) # Garde le label brut si non trouvé
            
            # Affichage des résultats
            print(f"--- RÉSULTATS : {nom_modele} ---")
            print(f"Statut : {friendly_label}")
            print(f"Confiance : {score:.2f}%")
            
            # Barre de progression visuelle
            bar_length = int(score / 5)
            print(f"Score : [{'█' * bar_length}{'-' * (20 - bar_length)}]")
            
        except Exception as e:
            print(f"❌ Erreur lors de l'analyse : {e}")

# --- LANCEMENT ---
analyze_button.on_click(on_button_clicked)
display(widgets.VBox([
    widgets.HTML("<h2>🛡️ FactGuard - Testeur Local multi-modèles</h2>"),
    model_dropdown, 
    text_input, 
    analyze_button, 
    output_area
]))

✅ Environnement chargé avec succès !
